In [1]:
import re
import pandas as pd

from nltk.tokenize import RegexpTokenizer

!pip install rouge
from rouge import Rouge 

from src.config import mturk_simjudgement_file, mturk_substitution_rating_scores

import warnings
warnings.filterwarnings('ignore')

In [2]:
tokenizer = RegexpTokenizer(r'\w+')
def clean_text(s):
    try:
        return ' '.join(tokenizer.tokenize(re.sub(r'[^a-zA-Z ]',r'', s).lower()))
    except:
        print(s)
        return ''

rouge = Rouge()

def get_rouge_scores(single_cands, reference):
    s = rouge.get_scores(clean_text(single_cands), clean_text(reference))[0]
    lst = []
    for m0 in ['rouge-1', 'rouge-2', 'rouge-l']:
        for m in ['f', 'p', 'r']: 
            lst.append(s[m0][m])
    return lst
                
def rouge_scores(line):
    single_cands = line['rec_item']
    multi_refs = line['gt_item']
    return get_rouge_scores(single_cands, multi_refs)


In [3]:
df = pd.read_csv(mturk_simjudgement_file)

df['ROUGE-1_f'], df['ROUGE-1_p'], df['ROUGE-1_r'], df['ROUGE-2_f'], \
 df['ROUGE-2_p'], df['ROUGE-2_r'], df['ROUGE-L_f'], df['ROUGE-L_p'], \
 df['ROUGE-L_r'] = zip(*df.apply(rouge_scores, axis=1))

cols = ['gt_item', 'rec_item', 'gt_item_id', 'rec_item_id', 'hint', 'SimJudgement', 
        'ROUGE-1_r','ROUGE-2_r', 'ROUGE-L_r']
df = df[cols].rename(columns={c:c.replace('_r', '') for c in cols})

df.to_csv(mturk_substitution_rating_scores['ROUGE'], index=True)

In [4]:
df.corr().iloc[3:4, 4:]

,ROUGE-1,ROUGE-2,ROUGE-L
SimJudgement,0.406427,0.360909,0.40276
